In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
directory = "/media/sony/0CA83E99A83E80EE/Chris/Bocconi/Thesis/Data_copy/Data/"

In [3]:
df_main = pd.read_csv(directory + 'base_affiliatesV2.csv' )

In [4]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77900 entries, 0 to 77899
Data columns (total 44 columns):
home_code            77900 non-null object
host_code            77900 non-null object
year                 77900 non-null int64
industry             77900 non-null int64
distance             75620 non-null float64
scoreproced          41980 non-null float64
minorinvest          41980 non-null float64
enfcontract          47720 non-null float64
paytax               41980 non-null float64
minorinvest_lag      38400 non-null float64
uncertainty          32200 non-null float64
home_gdp             73800 non-null float64
host_gdp             67720 non-null float64
couprisk             61200 non-null float64
couprisk_lag         61160 non-null float64
tariff_avg           50900 non-null float64
tariff_w_avg         50900 non-null float64
uncertainty_lag      29900 non-null float64
home_gdp_capita      73800 non-null float64
host_gdp_capita      67720 non-null float64
partner_directed 

In [5]:
code_map = pd.read_csv(directory + 'country_code_map.csv')

In [6]:
country_code_map = code_map.set_index('country').code.to_dict()

In [7]:
country_code_map['Hong Kong, China'] = 'HKG'

In [8]:
add_map = {'Russia': 'RUS', 'Venezuela':'VEN','Hong Kong': 'HKG', 'Taiwan': 'TWN' }

In [9]:
country_code_map.update(add_map)

In [10]:
col_names = ['country',1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]

In [11]:
df_concat = pd.DataFrame(columns = ['country','industry', 'aff_sales', 'host_code', 'year'])

for year in range(2009,2018):
    df_sales = pd.read_excel(directory + 'mofas-sales.xls', skiprows = 7, names=col_names, sheet_name = str(year))
    df_clean = df_sales.dropna(how = 'all', thresh =3).replace('(*)',0.0).replace('(D)', np.nan)
    df_long = pd.melt(df_clean,id_vars = 'country',var_name = 'industry',value_name ='aff_sales')
    df_long['host_code'] = df_long.country.map(country_code_map)
    df_long.dropna(subset= ['host_code'], inplace = True)
    df_long['year'] = int(year)
    df_concat = pd.concat([df_concat,df_long], sort = False)
    

In [12]:
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8064 entries, 0 to 1180
Data columns (total 5 columns):
country      8064 non-null object
industry     8064 non-null object
aff_sales    6747 non-null float64
host_code    8064 non-null object
year         8064 non-null object
dtypes: float64(1), object(4)
memory usage: 378.0+ KB


In [13]:
sales_df = df_concat[['host_code','year','industry','aff_sales']]

In [14]:
sales_df.head()

,host_code,year,industry,aff_sales
0,CAN,2009,1,505833.0
2,AUT,2009,1,20984.0
3,BEL,2009,1,117077.0
4,CZE,2009,1,16715.0
5,DNK,2009,1,18527.0


In [15]:
sales_df.tail()

,host_code,year,industry,aff_sales
1176,NZL,2017,16,1593.0
1177,PHL,2017,16,NaN
1178,SGP,2017,16,NaN
1179,TWN,2017,16,1495.0
1180,THA,2017,16,NaN


In [16]:
sales_df.to_csv(directory + 'affiliate_sales_clean.csv', index = False)

In [17]:
df_main = pd.merge(df_main, sales_df, how='left', on=['host_code','year', 'industry'])

In [19]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77900 entries, 0 to 77899
Data columns (total 45 columns):
home_code            77900 non-null object
host_code            77900 non-null object
year                 77900 non-null object
industry             77900 non-null object
distance             75620 non-null float64
scoreproced          41980 non-null float64
minorinvest          41980 non-null float64
enfcontract          47720 non-null float64
paytax               41980 non-null float64
minorinvest_lag      38400 non-null float64
uncertainty          32200 non-null float64
home_gdp             73800 non-null float64
host_gdp             67720 non-null float64
couprisk             61200 non-null float64
couprisk_lag         61160 non-null float64
tariff_avg           50900 non-null float64
tariff_w_avg         50900 non-null float64
uncertainty_lag      29900 non-null float64
home_gdp_capita      73800 non-null float64
host_gdp_capita      67720 non-null float64
partner_directe

In [20]:
df_main[df_main.aff_sales.notna()].head()

,home_code,host_code,year,industry,distance,scoreproced,minorinvest,enfcontract,paytax,minorinvest_lag,...,integrity_score,monthly_precip,yearly_precip,precip_volatility,road_home,road_host,home_polcon,host_polcon,border,aff_sales
2460,USA,ARE,2009,1,11025.539495,48.0,40.0,51.83,93.89,40.0,...,57.0,212.815385,2553.784615,291476.953077,NaN,NaN,0.398536,NaN,0.0,18271.0
2461,USA,ARE,2009,2,11025.539495,48.0,40.0,51.83,93.89,40.0,...,57.0,212.815385,2553.784615,291476.953077,NaN,NaN,0.398536,NaN,0.0,4389.0
2462,USA,ARE,2009,3,11025.539495,48.0,40.0,51.83,93.89,40.0,...,57.0,212.815385,2553.784615,291476.953077,NaN,NaN,0.398536,NaN,0.0,2608.0
2464,USA,ARE,2009,5,11025.539495,48.0,40.0,51.83,93.89,40.0,...,57.0,212.815385,2553.784615,291476.953077,NaN,NaN,0.398536,NaN,0.0,555.0
2467,USA,ARE,2009,8,11025.539495,48.0,40.0,51.83,93.89,40.0,...,57.0,212.815385,2553.784615,291476.953077,NaN,NaN,0.398536,NaN,0.0,307.0


In [21]:
df_concat = pd.DataFrame(columns = ['country','industry', 'aff_ex', 'host_code', 'year'])

for year in range(2009,2018):
    df_x = pd.read_excel(directory + 'affiliateXM_' +str(year) +'.xls', sheet_name ='Table II.H 3', skiprows = 9,
                  names=col_names)
    df_clean = df_x.dropna(how = 'all', thresh =3).replace('(*)',0.0).replace('(D)', np.nan)
    df_long = pd.melt(df_clean,id_vars = 'country',var_name = 'industry',value_name ='aff_ex')
    df_long['host_code'] = df_long.country.map(country_code_map)
    df_long.dropna(subset= ['host_code'], inplace = True)
    df_long['year'] = int(year)
    df_concat = pd.concat([df_concat,df_long], sort = False)
    

In [22]:
df_aff_ex = df_concat[['host_code','year','industry', 'aff_ex']]
df_aff_ex.head()

,host_code,year,industry,aff_ex
0,CAN,2009,1,63859.0
2,AUT,2009,1,233.0
3,BEL,2009,1,4470.0
4,CZE,2009,1,127.0
5,DNK,2009,1,300.0


In [23]:
df_aff_ex.tail()

,host_code,year,industry,aff_ex
1176,NZL,2017,16,0.0
1177,PHL,2017,16,1.0
1178,SGP,2017,16,NaN
1179,TWN,2017,16,0.0
1180,THA,2017,16,NaN


In [24]:
df_aff_ex.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8064 entries, 0 to 1180
Data columns (total 4 columns):
host_code    8064 non-null object
year         8064 non-null object
industry     8064 non-null object
aff_ex       6126 non-null float64
dtypes: float64(1), object(3)
memory usage: 315.0+ KB


In [25]:
df_aff_ex.to_csv(directory + 'affiliate_export_clean.csv', index = False)

In [26]:
df_main = pd.merge(df_main, df_aff_ex, how ='left', on =['host_code','year','industry'])

In [27]:
df_main.head()

,home_code,host_code,year,industry,distance,scoreproced,minorinvest,enfcontract,paytax,minorinvest_lag,...,monthly_precip,yearly_precip,precip_volatility,road_home,road_host,home_polcon,host_polcon,border,aff_sales,aff_ex
0,USA,ABW,2000,1,3147.434733,NaN,NaN,NaN,NaN,NaN,...,207.4,2488.8,290181.335,63.0,NaN,0.398317,NaN,0.0,NaN,NaN
1,USA,ABW,2000,2,3147.434733,NaN,NaN,NaN,NaN,NaN,...,207.4,2488.8,290181.335,63.0,NaN,0.398317,NaN,0.0,NaN,NaN
2,USA,ABW,2000,3,3147.434733,NaN,NaN,NaN,NaN,NaN,...,207.4,2488.8,290181.335,63.0,NaN,0.398317,NaN,0.0,NaN,NaN
3,USA,ABW,2000,4,3147.434733,NaN,NaN,NaN,NaN,NaN,...,207.4,2488.8,290181.335,63.0,NaN,0.398317,NaN,0.0,NaN,NaN
4,USA,ABW,2000,5,3147.434733,NaN,NaN,NaN,NaN,NaN,...,207.4,2488.8,290181.335,63.0,NaN,0.398317,NaN,0.0,NaN,NaN


In [28]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77900 entries, 0 to 77899
Data columns (total 46 columns):
home_code            77900 non-null object
host_code            77900 non-null object
year                 77900 non-null object
industry             77900 non-null object
distance             75620 non-null float64
scoreproced          41980 non-null float64
minorinvest          41980 non-null float64
enfcontract          47720 non-null float64
paytax               41980 non-null float64
minorinvest_lag      38400 non-null float64
uncertainty          32200 non-null float64
home_gdp             73800 non-null float64
host_gdp             67720 non-null float64
couprisk             61200 non-null float64
couprisk_lag         61160 non-null float64
tariff_avg           50900 non-null float64
tariff_w_avg         50900 non-null float64
uncertainty_lag      29900 non-null float64
home_gdp_capita      73800 non-null float64
host_gdp_capita      67720 non-null float64
partner_directe

In [29]:
df_concat = pd.DataFrame(columns = ['country','industry', 'aff_im', 'host_code', 'year'])

for year in range(2009,2018):
    df_x = pd.read_excel(directory + 'affiliateXM_' +str(year) +'.xls', sheet_name ='Table II.H 11', skiprows = 9,
                  names=col_names)
    df_clean = df_x.dropna(how = 'all', thresh =3).replace('(*)',0.0).replace('(D)', np.nan)
    df_long = pd.melt(df_clean,id_vars = 'country',var_name = 'industry',value_name ='aff_im')
    df_long['host_code'] = df_long.country.map(country_code_map)
    df_long.dropna(subset= ['host_code'], inplace = True)
    df_long['year'] = int(year)
    df_concat = pd.concat([df_concat,df_long], sort = False)

In [30]:
df_aff_im = df_concat[['host_code','year','industry', 'aff_im']]

In [31]:
df_aff_im.head()

,host_code,year,industry,aff_im
0,CAN,2009,1,83659.0
2,AUT,2009,1,200.0
3,BEL,2009,1,3378.0
4,CZE,2009,1,201.0
5,DNK,2009,1,296.0


In [32]:
df_aff_im.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8064 entries, 0 to 1180
Data columns (total 4 columns):
host_code    8064 non-null object
year         8064 non-null object
industry     8064 non-null object
aff_im       6100 non-null float64
dtypes: float64(1), object(3)
memory usage: 315.0+ KB


In [33]:
df_main = pd.merge(df_main, df_aff_im, how = 'left', on =['host_code','year', 'industry'])

In [35]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77900 entries, 0 to 77899
Data columns (total 47 columns):
home_code            77900 non-null object
host_code            77900 non-null object
year                 77900 non-null object
industry             77900 non-null object
distance             75620 non-null float64
scoreproced          41980 non-null float64
minorinvest          41980 non-null float64
enfcontract          47720 non-null float64
paytax               41980 non-null float64
minorinvest_lag      38400 non-null float64
uncertainty          32200 non-null float64
home_gdp             73800 non-null float64
host_gdp             67720 non-null float64
couprisk             61200 non-null float64
couprisk_lag         61160 non-null float64
tariff_avg           50900 non-null float64
tariff_w_avg         50900 non-null float64
uncertainty_lag      29900 non-null float64
home_gdp_capita      73800 non-null float64
host_gdp_capita      67720 non-null float64
partner_directe

In [36]:
df_concat = pd.DataFrame(columns = ['country','industry', 'aff_parex', 'host_code', 'year'])

for year in range(2009,2018):
    df_x = pd.read_excel(directory + 'affiliateXM_' +str(year) +'.xls', sheet_name ='Table II.H 5', skiprows = 9,
                  names=col_names)
    df_clean = df_x.dropna(how = 'all', thresh =3).replace('(*)',0.0).replace('(D)', np.nan)
    df_long = pd.melt(df_clean,id_vars = 'country',var_name = 'industry',value_name ='aff_parex')
    df_long['host_code'] = df_long.country.map(country_code_map)
    df_long.dropna(subset= ['host_code'], inplace = True)
    df_long['year'] = int(year)
    df_concat = pd.concat([df_concat,df_long], sort = False)
    
df_aff_parex = df_concat[['host_code','year','industry', 'aff_parex']]

In [37]:
df_aff_parex.head()

,host_code,year,industry,aff_parex
0,CAN,2009,1,52737.0
2,AUT,2009,1,216.0
3,BEL,2009,1,4079.0
4,CZE,2009,1,62.0
5,DNK,2009,1,298.0


In [38]:
df_main = pd.merge(df_main, df_aff_parex, how = 'left', on =['host_code','year', 'industry'])

In [40]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77900 entries, 0 to 77899
Data columns (total 48 columns):
home_code            77900 non-null object
host_code            77900 non-null object
year                 77900 non-null object
industry             77900 non-null object
distance             75620 non-null float64
scoreproced          41980 non-null float64
minorinvest          41980 non-null float64
enfcontract          47720 non-null float64
paytax               41980 non-null float64
minorinvest_lag      38400 non-null float64
uncertainty          32200 non-null float64
home_gdp             73800 non-null float64
host_gdp             67720 non-null float64
couprisk             61200 non-null float64
couprisk_lag         61160 non-null float64
tariff_avg           50900 non-null float64
tariff_w_avg         50900 non-null float64
uncertainty_lag      29900 non-null float64
home_gdp_capita      73800 non-null float64
host_gdp_capita      67720 non-null float64
partner_directe

In [41]:
df_concat = pd.DataFrame(columns = ['country','industry', 'aff_parim', 'host_code', 'year'])

for year in range(2009,2018):
    df_x = pd.read_excel(directory + 'affiliateXM_' +str(year) +'.xls', sheet_name ='Table II.H 13', skiprows = 9,
                  names=col_names)
    df_clean = df_x.dropna(how = 'all', thresh =3).replace('(*)',0.0).replace('(D)', np.nan)
    df_long = pd.melt(df_clean,id_vars = 'country',var_name = 'industry',value_name ='aff_parim')
    df_long['host_code'] = df_long.country.map(country_code_map)
    df_long.dropna(subset= ['host_code'], inplace = True)
    df_long['year'] = int(year)
    df_concat = pd.concat([df_concat,df_long], sort = False)
    
df_aff_parim = df_concat[['host_code','year','industry', 'aff_parim']]

In [42]:
df_aff_parim.head()

,host_code,year,industry,aff_parim
0,CAN,2009,1,68818.0
2,AUT,2009,1,160.0
3,BEL,2009,1,2884.0
4,CZE,2009,1,169.0
5,DNK,2009,1,245.0


In [43]:
df_main = pd.merge(df_main, df_aff_parim, how = 'left', on =['host_code','year', 'industry'])

In [44]:
df_main.head()

,home_code,host_code,year,industry,distance,scoreproced,minorinvest,enfcontract,paytax,minorinvest_lag,...,road_home,road_host,home_polcon,host_polcon,border,aff_sales,aff_ex,aff_im,aff_parex,aff_parim
0,USA,ABW,2000,1,3147.434733,NaN,NaN,NaN,NaN,NaN,...,63.0,NaN,0.398317,NaN,0.0,NaN,NaN,NaN,NaN,NaN
1,USA,ABW,2000,2,3147.434733,NaN,NaN,NaN,NaN,NaN,...,63.0,NaN,0.398317,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2,USA,ABW,2000,3,3147.434733,NaN,NaN,NaN,NaN,NaN,...,63.0,NaN,0.398317,NaN,0.0,NaN,NaN,NaN,NaN,NaN
3,USA,ABW,2000,4,3147.434733,NaN,NaN,NaN,NaN,NaN,...,63.0,NaN,0.398317,NaN,0.0,NaN,NaN,NaN,NaN,NaN
4,USA,ABW,2000,5,3147.434733,NaN,NaN,NaN,NaN,NaN,...,63.0,NaN,0.398317,NaN,0.0,NaN,NaN,NaN,NaN,NaN


In [45]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77900 entries, 0 to 77899
Data columns (total 49 columns):
home_code            77900 non-null object
host_code            77900 non-null object
year                 77900 non-null object
industry             77900 non-null object
distance             75620 non-null float64
scoreproced          41980 non-null float64
minorinvest          41980 non-null float64
enfcontract          47720 non-null float64
paytax               41980 non-null float64
minorinvest_lag      38400 non-null float64
uncertainty          32200 non-null float64
home_gdp             73800 non-null float64
host_gdp             67720 non-null float64
couprisk             61200 non-null float64
couprisk_lag         61160 non-null float64
tariff_avg           50900 non-null float64
tariff_w_avg         50900 non-null float64
uncertainty_lag      29900 non-null float64
home_gdp_capita      73800 non-null float64
host_gdp_capita      67720 non-null float64
partner_directe

In [46]:
df_main[df_main.aff_parim.notna()].head()

,home_code,host_code,year,industry,distance,scoreproced,minorinvest,enfcontract,paytax,minorinvest_lag,...,road_home,road_host,home_polcon,host_polcon,border,aff_sales,aff_ex,aff_im,aff_parex,aff_parim
2463,USA,ARE,2009,4,11025.539495,48.0,40.0,51.83,93.89,40.0,...,NaN,NaN,0.398536,NaN,0.0,NaN,NaN,0.0,NaN,0.0
2464,USA,ARE,2009,5,11025.539495,48.0,40.0,51.83,93.89,40.0,...,NaN,NaN,0.398536,NaN,0.0,555.0,NaN,0.0,NaN,0.0
2465,USA,ARE,2009,6,11025.539495,48.0,40.0,51.83,93.89,40.0,...,NaN,NaN,0.398536,NaN,0.0,NaN,1.0,0.0,1.0,0.0
2467,USA,ARE,2009,8,11025.539495,48.0,40.0,51.83,93.89,40.0,...,NaN,NaN,0.398536,NaN,0.0,307.0,NaN,0.0,NaN,0.0
2468,USA,ARE,2009,9,11025.539495,48.0,40.0,51.83,93.89,40.0,...,NaN,NaN,0.398536,NaN,0.0,NaN,0.0,0.0,0.0,0.0


In [47]:
df_main[df_main.aff_parim.notna()].head()

,home_code,host_code,year,industry,distance,scoreproced,minorinvest,enfcontract,paytax,minorinvest_lag,...,road_home,road_host,home_polcon,host_polcon,border,aff_sales,aff_ex,aff_im,aff_parex,aff_parim
2463,USA,ARE,2009,4,11025.539495,48.0,40.0,51.83,93.89,40.0,...,NaN,NaN,0.398536,NaN,0.0,NaN,NaN,0.0,NaN,0.0
2464,USA,ARE,2009,5,11025.539495,48.0,40.0,51.83,93.89,40.0,...,NaN,NaN,0.398536,NaN,0.0,555.0,NaN,0.0,NaN,0.0
2465,USA,ARE,2009,6,11025.539495,48.0,40.0,51.83,93.89,40.0,...,NaN,NaN,0.398536,NaN,0.0,NaN,1.0,0.0,1.0,0.0
2467,USA,ARE,2009,8,11025.539495,48.0,40.0,51.83,93.89,40.0,...,NaN,NaN,0.398536,NaN,0.0,307.0,NaN,0.0,NaN,0.0
2468,USA,ARE,2009,9,11025.539495,48.0,40.0,51.83,93.89,40.0,...,NaN,NaN,0.398536,NaN,0.0,NaN,0.0,0.0,0.0,0.0


In [48]:
fdi_cols20002002  = ['country',1,2,20,3,4,5,6,7,8,9,10,11,13,18,14,15,16]
fdi_cols20032018 = ['country', 1,2,3,4,5,6,7,8,9,10,17, 11,13, 18, 14,15,19,16]


In [49]:
df_concat = pd.DataFrame(columns = ['country','industry', 'fdi_flow', 'host_code', 'year'])

for year in range(2000,2019):
    if year in range(2000,2003):
        df_USfdi = pd.read_excel(directory + 'USFinTransac_2000_2009.xls', sheet_name = str(year), 
                                 skiprows = 6, names = fdi_cols20002002 , na_values = ' ')
    elif year in range(2003,2010):
        df_USfdi = pd.read_excel(directory + 'USFinTransac_2000_2009.xls', sheet_name = str(year), 
                                 skiprows = 6, names = fdi_cols20032018 , na_values = ' ')
    else:
        df_USfdi = pd.read_excel(directory + 'USFinTransac_2010_2018.xlsx', sheet_name = str(year), 
                                 skiprows = 6, names = fdi_cols20032018 , na_values = ' ')
        
    df_clean = df_USfdi.dropna(how = 'all', thresh =3).replace('(*)',0.0).replace('(D)', np.nan)
    df_long = pd.melt(df_clean,id_vars = 'country',var_name = 'industry',value_name ='fdi_flow')
    df_long['host_code'] = df_long.country.map(country_code_map)
    df_long.dropna(subset= ['host_code'], inplace = True)
    
    df_long['year'] = int(year)
    df_concat = pd.concat([df_concat,df_long], sort = False)
    
df_fdi_flow = df_concat[['host_code','year','industry', 'fdi_flow']]

In [50]:
df_fdi_flow.fdi_flow = df_fdi_flow.fdi_flow.apply(lambda x: float(x.replace(',','')) if isinstance(x, str) else x)

/home/sony/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [57]:
len(df_fdi_flow[['host_code','year','industry']])

18696

In [58]:
len(df_fdi_flow[['host_code','year','industry']].drop_duplicates() )

18696

In [59]:
df_fdi_flow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18696 entries, 0 to 1310
Data columns (total 4 columns):
host_code    18696 non-null object
year         18696 non-null object
industry     18696 non-null object
fdi_flow     14516 non-null float64
dtypes: float64(1), object(3)
memory usage: 730.3+ KB


In [60]:
df_main = pd.merge(df_main, df_fdi_flow, how ='left', on = ['host_code', 'year','industry'])

In [61]:
df_main.head()

,home_code,host_code,year,industry,distance,scoreproced,minorinvest,enfcontract,paytax,minorinvest_lag,...,road_host,home_polcon,host_polcon,border,aff_sales,aff_ex,aff_im,aff_parex,aff_parim,fdi_flow
0,USA,ABW,2000,1,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,0.398317,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,USA,ABW,2000,2,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,0.398317,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,USA,ABW,2000,3,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,0.398317,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,USA,ABW,2000,4,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,0.398317,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,USA,ABW,2000,5,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,0.398317,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77900 entries, 0 to 77899
Data columns (total 50 columns):
home_code            77900 non-null object
host_code            77900 non-null object
year                 77900 non-null object
industry             77900 non-null object
distance             75620 non-null float64
scoreproced          41980 non-null float64
minorinvest          41980 non-null float64
enfcontract          47720 non-null float64
paytax               41980 non-null float64
minorinvest_lag      38400 non-null float64
uncertainty          32200 non-null float64
home_gdp             73800 non-null float64
host_gdp             67720 non-null float64
couprisk             61200 non-null float64
couprisk_lag         61160 non-null float64
tariff_avg           50900 non-null float64
tariff_w_avg         50900 non-null float64
uncertainty_lag      29900 non-null float64
home_gdp_capita      73800 non-null float64
host_gdp_capita      67720 non-null float64
partner_directe

In [63]:
df_concat = pd.DataFrame(columns = ['country','industry', 'fdi_position', 'host_code', 'year'])

for year in range(2000,2019):
    if year in range(2000,2003):
        df_USfdi = pd.read_excel(directory + 'USFDI_Pos_2000_2009.xls', sheet_name = str(year), 
                                 skiprows = 6, names = fdi_cols20002002 , na_values = ' ')
    elif year in range(2003,2010):
        df_USfdi = pd.read_excel(directory + 'USFDI_Pos_2000_2009.xls', sheet_name = str(year), 
                                 skiprows = 6, names = fdi_cols20032018 , na_values = ' ')
    else:
        df_USfdi = pd.read_excel(directory + 'USFDI_Pos_2010_2018.xlsx', sheet_name = str(year), 
                                 skiprows = 6, names = fdi_cols20032018 , na_values = ' ')
        
    df_clean = df_USfdi.dropna(how = 'all', thresh =3).replace('(*)',0.0).replace('(D)', np.nan)
    df_long = pd.melt(df_clean,id_vars = 'country',var_name = 'industry',value_name ='fdi_position')
    df_long['host_code'] = df_long.country.map(country_code_map)
    df_long.dropna(subset= ['host_code'], inplace = True)
    
    df_long['year'] = int(year)
    df_concat = pd.concat([df_concat,df_long], sort = False)
    
df_fdi_position = df_concat[['host_code','year','industry', 'fdi_position']]

In [64]:
df_fdi_position.fdi_position = df_fdi_position.fdi_position.apply(lambda x: float(x.replace(',','')) if isinstance(x, str) else x)

In [66]:
df_fdi_position.head()

,host_code,year,industry,fdi_position
0,CAN,2000,1,132472.0
2,AUT,2000,1,2872.0
3,BEL,2000,1,17973.0
4,CZE,2000,1,1228.0
5,DNK,2000,1,5270.0


In [275]:
df_fdi_position.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18696 entries, 0 to 1310
Data columns (total 4 columns):
host_code       18696 non-null object
year            18696 non-null object
industry        18696 non-null object
fdi_position    15254 non-null float64
dtypes: float64(1), object(3)
memory usage: 730.3+ KB


In [68]:
len(df_fdi_position[['host_code','year','industry']] )

18696

In [69]:
len(df_fdi_position[['host_code','year','industry']].drop_duplicates())

18696

In [70]:
df_main = pd.merge(df_main, df_fdi_position, how = 'left', on =['host_code','year','industry'])

In [72]:
df_main.head()

,home_code,host_code,year,industry,distance,scoreproced,minorinvest,enfcontract,paytax,minorinvest_lag,...,home_polcon,host_polcon,border,aff_sales,aff_ex,aff_im,aff_parex,aff_parim,fdi_flow,fdi_position
0,USA,ABW,2000,1,3147.434733,NaN,NaN,NaN,NaN,NaN,...,0.398317,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,USA,ABW,2000,2,3147.434733,NaN,NaN,NaN,NaN,NaN,...,0.398317,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,USA,ABW,2000,3,3147.434733,NaN,NaN,NaN,NaN,NaN,...,0.398317,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,USA,ABW,2000,4,3147.434733,NaN,NaN,NaN,NaN,NaN,...,0.398317,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,USA,ABW,2000,5,3147.434733,NaN,NaN,NaN,NaN,NaN,...,0.398317,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77900 entries, 0 to 77899
Data columns (total 51 columns):
home_code            77900 non-null object
host_code            77900 non-null object
year                 77900 non-null object
industry             77900 non-null object
distance             75620 non-null float64
scoreproced          41980 non-null float64
minorinvest          41980 non-null float64
enfcontract          47720 non-null float64
paytax               41980 non-null float64
minorinvest_lag      38400 non-null float64
uncertainty          32200 non-null float64
home_gdp             73800 non-null float64
host_gdp             67720 non-null float64
couprisk             61200 non-null float64
couprisk_lag         61160 non-null float64
tariff_avg           50900 non-null float64
tariff_w_avg         50900 non-null float64
uncertainty_lag      29900 non-null float64
home_gdp_capita      73800 non-null float64
host_gdp_capita      67720 non-null float64
partner_directe

In [74]:
df_main.columns

Index(['home_code', 'host_code', 'year', 'industry', 'distance', 'scoreproced',
       'minorinvest', 'enfcontract', 'paytax', 'minorinvest_lag',
       'uncertainty', 'home_gdp', 'host_gdp', 'couprisk', 'couprisk_lag',
       'tariff_avg', 'tariff_w_avg', 'uncertainty_lag', 'home_gdp_capita',
       'host_gdp_capita', 'partner_directed', 'OECD_partner', 'home_north',
       'host_north', 'north-north', 'north-south', 'south-north',
       'south-south', 'host_lock', 'lang', 'NAFTA', 'migration',
       'fiscal_score', 'invest_score', 'judic_score', 'integrity_score',
       'monthly_precip', 'yearly_precip', 'precip_volatility', 'road_home',
       'road_host', 'home_polcon', 'host_polcon', 'border', 'aff_sales',
       'aff_ex', 'aff_im', 'aff_parex', 'aff_parim', 'fdi_flow',
       'fdi_position'],
      dtype='object')

In [75]:
df_main[['aff_sales',
       'aff_ex', 'aff_im', 'aff_parex', 'aff_parim', 'fdi_flow',
       'fdi_position']] = df_main[['aff_sales',
       'aff_ex', 'aff_im', 'aff_parex', 'aff_parim', 'fdi_flow',
       'fdi_position']] * 1_000_000

In [76]:
df_main[df_main.fdi_position.notna()]

,home_code,host_code,year,industry,distance,scoreproced,minorinvest,enfcontract,paytax,minorinvest_lag,...,home_polcon,host_polcon,border,aff_sales,aff_ex,aff_im,aff_parex,aff_parim,fdi_flow,fdi_position
2280,USA,ARE,2000,1,11025.539495,NaN,NaN,NaN,NaN,NaN,...,0.398317,NaN,0.0,NaN,NaN,NaN,NaN,NaN,94000000.0,6.830000e+08
2283,USA,ARE,2000,4,11025.539495,NaN,NaN,NaN,NaN,NaN,...,0.398317,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.000000e+00
2285,USA,ARE,2000,6,11025.539495,NaN,NaN,NaN,NaN,NaN,...,0.398317,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.100000e+07
2286,USA,ARE,2000,7,11025.539495,NaN,NaN,NaN,NaN,NaN,...,0.398317,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.000000e+00
2287,USA,ARE,2000,8,11025.539495,NaN,NaN,NaN,NaN,NaN,...,0.398317,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.000000e+00
2288,USA,ARE,2000,9,11025.539495,NaN,NaN,NaN,NaN,NaN,...,0.398317,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.000000e+00
2289,USA,ARE,2000,10,11025.539495,NaN,NaN,NaN,NaN,NaN,...,0.398317,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.000000e+00
2290,USA,ARE,2000,11,11025.539495,NaN,NaN,NaN,NaN,NaN,...,0.398317,NaN,0.0,NaN,NaN,NaN,NaN,NaN,118000000.0,2.810000e+08
2292,USA,ARE,2000,13,11025.539495,NaN,NaN,NaN,NaN,NaN,...,0.398317,NaN,0.0,NaN,NaN,NaN,NaN,NaN,7000000.0,0.000000e+00
2293,USA,ARE,2000,14,11025.539495,NaN,NaN,NaN,NaN,NaN,...,0.398317,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.500000e+07


In [77]:
df_main.head()

,home_code,host_code,year,industry,distance,scoreproced,minorinvest,enfcontract,paytax,minorinvest_lag,...,home_polcon,host_polcon,border,aff_sales,aff_ex,aff_im,aff_parex,aff_parim,fdi_flow,fdi_position
0,USA,ABW,2000,1,3147.434733,NaN,NaN,NaN,NaN,NaN,...,0.398317,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,USA,ABW,2000,2,3147.434733,NaN,NaN,NaN,NaN,NaN,...,0.398317,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,USA,ABW,2000,3,3147.434733,NaN,NaN,NaN,NaN,NaN,...,0.398317,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,USA,ABW,2000,4,3147.434733,NaN,NaN,NaN,NaN,NaN,...,0.398317,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,USA,ABW,2000,5,3147.434733,NaN,NaN,NaN,NaN,NaN,...,0.398317,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77900 entries, 0 to 77899
Data columns (total 51 columns):
home_code            77900 non-null object
host_code            77900 non-null object
year                 77900 non-null object
industry             77900 non-null object
distance             75620 non-null float64
scoreproced          41980 non-null float64
minorinvest          41980 non-null float64
enfcontract          47720 non-null float64
paytax               41980 non-null float64
minorinvest_lag      38400 non-null float64
uncertainty          32200 non-null float64
home_gdp             73800 non-null float64
host_gdp             67720 non-null float64
couprisk             61200 non-null float64
couprisk_lag         61160 non-null float64
tariff_avg           50900 non-null float64
tariff_w_avg         50900 non-null float64
uncertainty_lag      29900 non-null float64
home_gdp_capita      73800 non-null float64
host_gdp_capita      67720 non-null float64
partner_directe

In [96]:
df_UStrade = pd.read_csv(directory + 'USexport_country_industry.csv', encoding = "ISO-8859-1")

In [97]:
df_UStrade.rename(columns = {'TradeValue in 1000 USD': 'Export'}, inplace = True)

In [98]:
df_UStrade.Export = df_UStrade.Export*1000

In [99]:
df_UStrade.head()

,Nomenclature,ReporterISO3,ProductCode,ReporterName,PartnerISO3,PartnerName,Year,TradeFlowName,TradeFlowCode,Export
0,H1,USA,Total,United States,ABW,Aruba,2000,Export,6,271792324.0
1,H1,USA,Total,United States,ABW,Aruba,2001,Export,6,260978060.0
2,H1,USA,Total,United States,ABW,Aruba,2002,Export,6,442454334.0
3,H1,USA,Total,United States,ABW,Aruba,2003,Export,6,317571789.0
4,H1,USA,Total,United States,ABW,Aruba,2004,Export,6,348288059.0


In [100]:
df_UStrade.ProductCode = df_UStrade.ProductCode.str.strip()

In [101]:
cat_map = {'Total': 1, '01-05_Animal': 4,  '06-15_Vegetable': 4, '16-24_FoodProd' : 4, '25-26_Minerals': 2,
          '27-27_Fuels': 2, '28-38_Chemicals': 5, '39-40_PlastiRub' : 'aa', '41-43_HidesSkin': 16, '44-49_Wood':16,
          '50-63_TextCloth': 'dd', '64-67_Footwear':'dd' , '68-71_StoneGlas':2, '72-83_Metals': 6,
           '84-85_MachElec': 9, '86-89_Transport': 10, '90-99_Miscellan': 'ee' }

In [102]:
df_UStrade['industry'] = df_UStrade.ProductCode.map(cat_map)

In [103]:
df_UStrade.head()

,Nomenclature,ReporterISO3,ProductCode,ReporterName,PartnerISO3,PartnerName,Year,TradeFlowName,TradeFlowCode,Export,industry
0,H1,USA,Total,United States,ABW,Aruba,2000,Export,6,271792324.0,1
1,H1,USA,Total,United States,ABW,Aruba,2001,Export,6,260978060.0,1
2,H1,USA,Total,United States,ABW,Aruba,2002,Export,6,442454334.0,1
3,H1,USA,Total,United States,ABW,Aruba,2003,Export,6,317571789.0,1
4,H1,USA,Total,United States,ABW,Aruba,2004,Export,6,348288059.0,1


In [104]:
df_UStrade = df_UStrade.groupby(['ReporterISO3','PartnerISO3', 'Year', 'industry'])['Export'].sum().to_frame().reset_index()

In [107]:
df_UStrade.head()

,ReporterISO3,PartnerISO3,Year,industry,Export
0,USA,ABW,2000,1,271792324.0
1,USA,ABW,2000,2,71186832.0
2,USA,ABW,2000,4,30969497.0
3,USA,ABW,2000,5,23017107.0
4,USA,ABW,2000,6,9709398.0


In [108]:
df_US_indusX = df_UStrade[['ReporterISO3', 'PartnerISO3', 'Year', 'industry', 'Export']]

In [109]:
df_US_indusX.columns = ['home_code','host_code','year', 'industry','indus_ex']

In [110]:
df_US_indusX.head()

,home_code,host_code,year,industry,indus_ex
0,USA,ABW,2000,1,271792324.0
1,USA,ABW,2000,2,71186832.0
2,USA,ABW,2000,4,30969497.0
3,USA,ABW,2000,5,23017107.0
4,USA,ABW,2000,6,9709398.0


In [111]:
df_main = pd.merge(df_main, df_US_indusX, how ='left', on = ['home_code', 'host_code', 'year', 'industry'])

In [112]:
df_main.head()

,home_code,host_code,year,industry,distance,scoreproced,minorinvest,enfcontract,paytax,minorinvest_lag,...,host_polcon,border,aff_sales,aff_ex,aff_im,aff_parex,aff_parim,fdi_flow,fdi_position,indus_ex
0,USA,ABW,2000,1,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271792324.0
1,USA,ABW,2000,2,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71186832.0
2,USA,ABW,2000,3,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,USA,ABW,2000,4,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30969497.0
4,USA,ABW,2000,5,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23017107.0


In [113]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77900 entries, 0 to 77899
Data columns (total 52 columns):
home_code            77900 non-null object
host_code            77900 non-null object
year                 77900 non-null object
industry             77900 non-null object
distance             75620 non-null float64
scoreproced          41980 non-null float64
minorinvest          41980 non-null float64
enfcontract          47720 non-null float64
paytax               41980 non-null float64
minorinvest_lag      38400 non-null float64
uncertainty          32200 non-null float64
home_gdp             73800 non-null float64
host_gdp             67720 non-null float64
couprisk             61200 non-null float64
couprisk_lag         61160 non-null float64
tariff_avg           50900 non-null float64
tariff_w_avg         50900 non-null float64
uncertainty_lag      29900 non-null float64
home_gdp_capita      73800 non-null float64
host_gdp_capita      67720 non-null float64
partner_directe

In [114]:
def mine_vs_man(x):
    if x  == 2 or x ==1:
        return 0
    else:
        return 1

In [115]:
df_UStrade['manufac'] = df_UStrade['industry'].apply(mine_vs_man)

In [118]:
df_UStrade.head(7)

,ReporterISO3,PartnerISO3,Year,industry,Export,manufac
0,USA,ABW,2000,1,271792324.0,0
1,USA,ABW,2000,2,71186832.0,0
2,USA,ABW,2000,4,30969497.0,1
3,USA,ABW,2000,5,23017107.0,1
4,USA,ABW,2000,6,9709398.0,1
5,USA,ABW,2000,9,64781721.0,1
6,USA,ABW,2000,10,12350053.0,1


In [119]:
df_US_Ex = df_UStrade[['ReporterISO3', 'PartnerISO3', 'Year', 'Export', 'industry', 'manufac' ]]

In [120]:
df_US_Ex.columns = ['home_code', 'host_code', 'year','export','industry', 'manufac']

In [126]:
df_US_Ex.groupby(['home_code', 'host_code', 'year','manufac']).export.sum().unstack('manufac')

manufac                              0             1
home_code host_code year                            
USA       ABW       2000  3.429792e+08  2.006055e+08
                    2001  3.187997e+08  2.031564e+08
                    2002  5.525952e+08  3.058217e+08
                    2003  4.116374e+08  2.026502e+08
                    2004  5.101161e+08  1.644052e+08
                    2005  7.073463e+08  2.482837e+08
                    2006  6.894749e+08  2.414426e+08
                    2007  6.728024e+08  2.916939e+08
                    2008  9.616830e+08  2.962022e+08
                    2009  5.660491e+08  2.426811e+08
                    2010  7.530498e+08  2.472931e+08
                    2011  9.875904e+08  3.307099e+08
                    2012  1.000026e+09  2.893433e+08
                    2013  1.732791e+09  3.128027e+08
                    2014  2.162203e+09  3.536331e+08
                    2015  1.887642e+09  3.146801e+08
                    2016  1.167168e+09  3.069927e+08
                    2017  1.410843e+09  3.475925e+08
                    2018  7.359402e+08  3.323033e+08
          AFG       2000  8.650166e+06  7.647298e+06
                    2001  5.809661e+06  5.628285e+06
                    2002  7.985301e+07  7.551582e+07
                    2003  5.859559e+07  5.491703e+07
                    2004  1.264685e+08  1.169022e+08
                    2005  2.523676e+08  2.403995e+08
                    2006  4.158778e+08  4.003070e+08
                    2007  4.748540e+08  4.711593e+08
                    2008  4.746498e+08  4.688319e+08
                    2009  1.476746e+09  1.467248e+09
                    2010  2.035751e+09  2.027837e+09
...                                ...           ...
          ZMB       2008  7.599534e+07  7.589478e+07
                    2009  5.669243e+07  5.666042e+07
                    2010  5.424550e+07  5.357154e+07
                    2011  1.238292e+08  1.236422e+08
                    2012  1.419712e+08  1.417563e+08
                    2013  1.313691e+08  1.310955e+08
                    2014  1.069161e+08  1.066961e+08
                    2015  7.714900e+07  7.659798e+07
                    2016  7.660875e+07  7.621702e+07
                    2017  7.904716e+07  7.858741e+07
                    2018  1.860156e+08  1.837081e+08
          ZWE       2000  5.196167e+07  5.188453e+07
                    2001  3.138030e+07  3.051757e+07
                    2002  5.288939e+07  4.331246e+07
                    2003  4.523868e+07  3.565313e+07
                    2004  4.807026e+07  4.056132e+07
                    2005  4.601530e+07  4.066812e+07
                    2006  4.737021e+07  4.401879e+07
                    2007  1.036129e+08  1.035315e+08
                    2008  9.148806e+07  9.133199e+07
                    2009  8.429021e+07  8.414281e+07
                    2010  6.547360e+07  6.536315e+07
                    2011  5.841138e+07  5.818427e+07
                    2012  4.738196e+07  4.702368e+07
                    2013  5.016392e+07  5.005683e+07
                    2014  4.097518e+07  4.083780e+07
                    2015  3.318994e+07  3.312358e+07
                    2016  2.618201e+07  2.616102e+07
                    2017  3.486014e+07  3.482941e+07
                    2018  3.091076e+07  3.088082e+07

[4248 rows x 2 columns]

In [127]:
total_manX = df_US_Ex.groupby(['home_code', 'host_code', 'year','manufac']).export.sum().unstack('manufac').loc[:,1].to_frame()

In [128]:
total_manX

1
home_code host_code year              
USA       ABW       2000  2.006055e+08
                    2001  2.031564e+08
                    2002  3.058217e+08
                    2003  2.026502e+08
                    2004  1.644052e+08
                    2005  2.482837e+08
                    2006  2.414426e+08
                    2007  2.916939e+08
                    2008  2.962022e+08
                    2009  2.426811e+08
                    2010  2.472931e+08
                    2011  3.307099e+08
                    2012  2.893433e+08
                    2013  3.128027e+08
                    2014  3.536331e+08
                    2015  3.146801e+08
                    2016  3.069927e+08
                    2017  3.475925e+08
                    2018  3.323033e+08
          AFG       2000  7.647298e+06
                    2001  5.628285e+06
                    2002  7.551582e+07
                    2003  5.491703e+07
                    2004  1.169022e+08
                    2005  2.403995e+08
                    2006  4.003070e+08
                    2007  4.711593e+08
                    2008  4.688319e+08
                    2009  1.467248e+09
                    2010  2.027837e+09
...                                ...
          ZMB       2008  7.589478e+07
                    2009  5.666042e+07
                    2010  5.357154e+07
                    2011  1.236422e+08
                    2012  1.417563e+08
                    2013  1.310955e+08
                    2014  1.066961e+08
                    2015  7.659798e+07
                    2016  7.621702e+07
                    2017  7.858741e+07
                    2018  1.837081e+08
          ZWE       2000  5.188453e+07
                    2001  3.051757e+07
                    2002  4.331246e+07
                    2003  3.565313e+07
                    2004  4.056132e+07
                    2005  4.066812e+07
                    2006  4.401879e+07
                    2007  1.035315e+08
                    2008  9.133199e+07
                    2009  8.414281e+07
                    2010  6.536315e+07
                    2011  5.818427e+07
                    2012  4.702368e+07
                    2013  5.005683e+07
                    2014  4.083780e+07
                    2015  3.312358e+07
                    2016  2.616102e+07
                    2017  3.482941e+07
                    2018  3.088082e+07

[4248 rows x 1 columns]

In [129]:
total_manX.columns = ['indus_ex']

In [130]:
total_manX.head()

indus_ex
home_code host_code year             
USA       ABW       2000  200605492.0
                    2001  203156396.0
                    2002  305821698.0
                    2003  202650174.0
                    2004  164405235.0

In [131]:
total_manX.reset_index(inplace = True)
total_manX['industry'] = int(3)
total_manX.head()

,home_code,host_code,year,indus_ex,industry
0,USA,ABW,2000,200605492.0,3
1,USA,ABW,2001,203156396.0,3
2,USA,ABW,2002,305821698.0,3
3,USA,ABW,2003,202650174.0,3
4,USA,ABW,2004,164405235.0,3


In [132]:
df_main = pd.merge(df_main, total_manX, how = 'left', on =['home_code','host_code','year', 'industry'])
df_main.head()

,home_code,host_code,year,industry,distance,scoreproced,minorinvest,enfcontract,paytax,minorinvest_lag,...,border,aff_sales,aff_ex,aff_im,aff_parex,aff_parim,fdi_flow,fdi_position,indus_ex_x,indus_ex_y
0,USA,ABW,2000,1,3147.434733,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271792324.0,NaN
1,USA,ABW,2000,2,3147.434733,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71186832.0,NaN
2,USA,ABW,2000,3,3147.434733,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200605492.0
3,USA,ABW,2000,4,3147.434733,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30969497.0,NaN
4,USA,ABW,2000,5,3147.434733,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23017107.0,NaN


In [133]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77900 entries, 0 to 77899
Data columns (total 53 columns):
home_code            77900 non-null object
host_code            77900 non-null object
year                 77900 non-null object
industry             77900 non-null object
distance             75620 non-null float64
scoreproced          41980 non-null float64
minorinvest          41980 non-null float64
enfcontract          47720 non-null float64
paytax               41980 non-null float64
minorinvest_lag      38400 non-null float64
uncertainty          32200 non-null float64
home_gdp             73800 non-null float64
host_gdp             67720 non-null float64
couprisk             61200 non-null float64
couprisk_lag         61160 non-null float64
tariff_avg           50900 non-null float64
tariff_w_avg         50900 non-null float64
uncertainty_lag      29900 non-null float64
home_gdp_capita      73800 non-null float64
host_gdp_capita      67720 non-null float64
partner_directe

In [134]:
def combine_indus_ex(x):
    if np.isnan(x[0]):
        return x[1]
    else:
        return x[0]

In [135]:
df_main['indus_ex'] = df_main[['indus_ex_x', 'indus_ex_y']].apply(lambda x: x[1] if np.isnan(x[0]) else x[0], axis =1)

In [136]:
df_main.head()

,home_code,host_code,year,industry,distance,scoreproced,minorinvest,enfcontract,paytax,minorinvest_lag,...,aff_sales,aff_ex,aff_im,aff_parex,aff_parim,fdi_flow,fdi_position,indus_ex_x,indus_ex_y,indus_ex
0,USA,ABW,2000,1,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271792324.0,NaN,271792324.0
1,USA,ABW,2000,2,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71186832.0,NaN,71186832.0
2,USA,ABW,2000,3,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200605492.0,200605492.0
3,USA,ABW,2000,4,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30969497.0,NaN,30969497.0
4,USA,ABW,2000,5,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23017107.0,NaN,23017107.0


In [137]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77900 entries, 0 to 77899
Data columns (total 54 columns):
home_code            77900 non-null object
host_code            77900 non-null object
year                 77900 non-null object
industry             77900 non-null object
distance             75620 non-null float64
scoreproced          41980 non-null float64
minorinvest          41980 non-null float64
enfcontract          47720 non-null float64
paytax               41980 non-null float64
minorinvest_lag      38400 non-null float64
uncertainty          32200 non-null float64
home_gdp             73800 non-null float64
host_gdp             67720 non-null float64
couprisk             61200 non-null float64
couprisk_lag         61160 non-null float64
tariff_avg           50900 non-null float64
tariff_w_avg         50900 non-null float64
uncertainty_lag      29900 non-null float64
home_gdp_capita      73800 non-null float64
host_gdp_capita      67720 non-null float64
partner_directe

In [138]:
df_main.drop(['indus_ex_x', 'indus_ex_y'], axis =1, inplace = True)

In [139]:
df_main.head()

,home_code,host_code,year,industry,distance,scoreproced,minorinvest,enfcontract,paytax,minorinvest_lag,...,host_polcon,border,aff_sales,aff_ex,aff_im,aff_parex,aff_parim,fdi_flow,fdi_position,indus_ex
0,USA,ABW,2000,1,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271792324.0
1,USA,ABW,2000,2,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71186832.0
2,USA,ABW,2000,3,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200605492.0
3,USA,ABW,2000,4,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30969497.0
4,USA,ABW,2000,5,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23017107.0


In [140]:
df_main.to_csv(directory + 'affiliates_data.csv', index = False)

In [141]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77900 entries, 0 to 77899
Data columns (total 52 columns):
home_code            77900 non-null object
host_code            77900 non-null object
year                 77900 non-null object
industry             77900 non-null object
distance             75620 non-null float64
scoreproced          41980 non-null float64
minorinvest          41980 non-null float64
enfcontract          47720 non-null float64
paytax               41980 non-null float64
minorinvest_lag      38400 non-null float64
uncertainty          32200 non-null float64
home_gdp             73800 non-null float64
host_gdp             67720 non-null float64
couprisk             61200 non-null float64
couprisk_lag         61160 non-null float64
tariff_avg           50900 non-null float64
tariff_w_avg         50900 non-null float64
uncertainty_lag      29900 non-null float64
home_gdp_capita      73800 non-null float64
host_gdp_capita      67720 non-null float64
partner_directe

In [142]:
df_UStrade = pd.read_csv(directory + 'USimport_country_industry.csv', encoding = "ISO-8859-1")

In [143]:
df_UStrade.rename(columns = {'TradeValue in 1000 USD': 'Import'}, inplace = True)

In [144]:
df_UStrade.Import = df_UStrade.Import*1000

In [145]:
df_UStrade.head()

,Nomenclature,ReporterISO3,ProductCode,ReporterName,PartnerISO3,PartnerName,Year,TradeFlowName,TradeFlowCode,Import
0,H1,USA,Total,United States,ABW,Aruba,2000,Import,5,1.535549e+09
1,H1,USA,Total,United States,ABW,Aruba,2001,Import,5,1.034035e+09
2,H1,USA,Total,United States,ABW,Aruba,2002,Import,5,8.056989e+08
3,H1,USA,Total,United States,ABW,Aruba,2003,Import,5,1.001703e+09
4,H1,USA,Total,United States,ABW,Aruba,2004,Import,5,1.862156e+09


In [146]:
df_UStrade.ProductCode = df_UStrade.ProductCode.str.strip()

In [147]:
df_UStrade.ProductCode.unique()

array(['Total', '01-05_Animal', '06-15_Vegetable', '16-24_FoodProd',
       '25-26_Minerals', '27-27_Fuels', '28-38_Chemicals',
       '39-40_PlastiRub', '41-43_HidesSkin', '44-49_Wood',
       '50-63_TextCloth', '64-67_Footwear', '68-71_StoneGlas',
       '72-83_Metals', '84-85_MachElec', '86-89_Transport',
       '90-99_Miscellan'], dtype=object)

In [148]:
df_UStrade['industry'] = df_UStrade.ProductCode.map(cat_map)

In [150]:
df_UStrade.head()

,Nomenclature,ReporterISO3,ProductCode,ReporterName,PartnerISO3,PartnerName,Year,TradeFlowName,TradeFlowCode,Import,industry
0,H1,USA,Total,United States,ABW,Aruba,2000,Import,5,1.535549e+09,1
1,H1,USA,Total,United States,ABW,Aruba,2001,Import,5,1.034035e+09,1
2,H1,USA,Total,United States,ABW,Aruba,2002,Import,5,8.056989e+08,1
3,H1,USA,Total,United States,ABW,Aruba,2003,Import,5,1.001703e+09,1
4,H1,USA,Total,United States,ABW,Aruba,2004,Import,5,1.862156e+09,1


In [152]:
df_UStrade = df_UStrade.groupby(['ReporterISO3','PartnerISO3', 'Year', 'industry'])['Import'].sum().to_frame().reset_index()

In [153]:
len(df_UStrade)

39278

In [154]:
len(df_UStrade.drop_duplicates())

39278

In [155]:
df_US_indusM = df_UStrade[['ReporterISO3', 'PartnerISO3', 'Year', 'industry', 'Import']]

In [156]:
df_US_indusM.columns = ['home_code','host_code','year', 'industry','indus_im']

In [157]:
df_US_indusM.head()

,home_code,host_code,year,industry,indus_im
0,USA,ABW,2000,1,1.535549e+09
1,USA,ABW,2000,2,1.461237e+09
2,USA,ABW,2000,4,1.302712e+06
3,USA,ABW,2000,5,3.524300e+04
4,USA,ABW,2000,6,7.750950e+05


In [158]:
df_main = pd.merge(df_main, df_US_indusM, how ='left', on = ['home_code', 'host_code', 'year', 'industry'])

In [159]:
df_main.head()

,home_code,host_code,year,industry,distance,scoreproced,minorinvest,enfcontract,paytax,minorinvest_lag,...,border,aff_sales,aff_ex,aff_im,aff_parex,aff_parim,fdi_flow,fdi_position,indus_ex,indus_im
0,USA,ABW,2000,1,3147.434733,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271792324.0,1.535549e+09
1,USA,ABW,2000,2,3147.434733,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71186832.0,1.461237e+09
2,USA,ABW,2000,3,3147.434733,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200605492.0,NaN
3,USA,ABW,2000,4,3147.434733,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30969497.0,1.302712e+06
4,USA,ABW,2000,5,3147.434733,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23017107.0,3.524300e+04


In [160]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77900 entries, 0 to 77899
Data columns (total 53 columns):
home_code            77900 non-null object
host_code            77900 non-null object
year                 77900 non-null object
industry             77900 non-null object
distance             75620 non-null float64
scoreproced          41980 non-null float64
minorinvest          41980 non-null float64
enfcontract          47720 non-null float64
paytax               41980 non-null float64
minorinvest_lag      38400 non-null float64
uncertainty          32200 non-null float64
home_gdp             73800 non-null float64
host_gdp             67720 non-null float64
couprisk             61200 non-null float64
couprisk_lag         61160 non-null float64
tariff_avg           50900 non-null float64
tariff_w_avg         50900 non-null float64
uncertainty_lag      29900 non-null float64
home_gdp_capita      73800 non-null float64
host_gdp_capita      67720 non-null float64
partner_directe

In [161]:
df_UStrade['manufac'] = df_UStrade['industry'].apply(mine_vs_man)

In [162]:
df_US_Im = df_UStrade[['ReporterISO3', 'PartnerISO3', 'Year', 'Import', 'industry', 'manufac' ]]

In [163]:
df_US_Im.columns = ['home_code', 'host_code', 'year','import','industry', 'manufac']

In [164]:
df_US_Im.head()

,home_code,host_code,year,import,industry,manufac
0,USA,ABW,2000,1.535549e+09,1,0
1,USA,ABW,2000,1.461237e+09,2,0
2,USA,ABW,2000,1.302712e+06,4,1
3,USA,ABW,2000,3.524300e+04,5,1
4,USA,ABW,2000,7.750950e+05,6,1


In [168]:
df_US_Im.head(12)

,home_code,host_code,year,import,industry,manufac
0,USA,ABW,2000,1.535549e+09,1,0
1,USA,ABW,2000,1.461237e+09,2,0
2,USA,ABW,2000,1.302712e+06,4,1
3,USA,ABW,2000,3.524300e+04,5,1
4,USA,ABW,2000,7.750950e+05,6,1
5,USA,ABW,2000,8.572900e+04,9,1
6,USA,ABW,2000,2.600000e+02,16,1
7,USA,ABW,2000,7.670000e+02,aa,1
8,USA,ABW,2000,7.400000e+02,dd,1
9,USA,ABW,2000,7.211160e+07,ee,1


In [169]:
total_manM = df_US_Im.groupby(['home_code', 'host_code', 'year','manufac'])['import'].sum().unstack('manufac').loc[:,1].to_frame()

In [170]:
total_manM.columns = ['indus_im']

In [171]:
total_manM.head()

indus_im
home_code host_code year            
USA       ABW       2000  74312151.0
                    2001  10511397.0
                    2002   7346854.0
                    2003   1387382.0
                    2004   2292478.0

In [172]:
total_manM.reset_index(inplace = True)
total_manM['industry'] = int(3)
total_manM.head()

,home_code,host_code,year,indus_im,industry
0,USA,ABW,2000,74312151.0,3
1,USA,ABW,2001,10511397.0,3
2,USA,ABW,2002,7346854.0,3
3,USA,ABW,2003,1387382.0,3
4,USA,ABW,2004,2292478.0,3


In [173]:
df_main = pd.merge(df_main, total_manM, how = 'left', on =['home_code','host_code','year', 'industry'])
df_main

,home_code,host_code,year,industry,distance,scoreproced,minorinvest,enfcontract,paytax,minorinvest_lag,...,aff_sales,aff_ex,aff_im,aff_parex,aff_parim,fdi_flow,fdi_position,indus_ex,indus_im_x,indus_im_y
0,USA,ABW,2000,1,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271792324.0,1.535549e+09,NaN
1,USA,ABW,2000,2,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71186832.0,1.461237e+09,NaN
2,USA,ABW,2000,3,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200605492.0,NaN,74312151.0
3,USA,ABW,2000,4,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30969497.0,1.302712e+06,NaN
4,USA,ABW,2000,5,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23017107.0,3.524300e+04,NaN
5,USA,ABW,2000,6,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9709398.0,7.750950e+05,NaN
6,USA,ABW,2000,7,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,USA,ABW,2000,8,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,USA,ABW,2000,9,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64781721.0,8.572900e+04,NaN
9,USA,ABW,2000,10,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12350053.0,NaN,NaN


In [174]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77900 entries, 0 to 77899
Data columns (total 54 columns):
home_code            77900 non-null object
host_code            77900 non-null object
year                 77900 non-null object
industry             77900 non-null object
distance             75620 non-null float64
scoreproced          41980 non-null float64
minorinvest          41980 non-null float64
enfcontract          47720 non-null float64
paytax               41980 non-null float64
minorinvest_lag      38400 non-null float64
uncertainty          32200 non-null float64
home_gdp             73800 non-null float64
host_gdp             67720 non-null float64
couprisk             61200 non-null float64
couprisk_lag         61160 non-null float64
tariff_avg           50900 non-null float64
tariff_w_avg         50900 non-null float64
uncertainty_lag      29900 non-null float64
home_gdp_capita      73800 non-null float64
host_gdp_capita      67720 non-null float64
partner_directe

In [175]:
df_main['indus_im'] = df_main[['indus_im_x', 'indus_im_y']].apply(lambda x: x[1] if np.isnan(x[0]) else x[0], axis =1)

In [176]:
df_main.head()

,home_code,host_code,year,industry,distance,scoreproced,minorinvest,enfcontract,paytax,minorinvest_lag,...,aff_ex,aff_im,aff_parex,aff_parim,fdi_flow,fdi_position,indus_ex,indus_im_x,indus_im_y,indus_im
0,USA,ABW,2000,1,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,271792324.0,1.535549e+09,NaN,1.535549e+09
1,USA,ABW,2000,2,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,71186832.0,1.461237e+09,NaN,1.461237e+09
2,USA,ABW,2000,3,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,200605492.0,NaN,74312151.0,7.431215e+07
3,USA,ABW,2000,4,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,30969497.0,1.302712e+06,NaN,1.302712e+06
4,USA,ABW,2000,5,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,23017107.0,3.524300e+04,NaN,3.524300e+04


In [177]:
df_main.drop(['indus_im_x', 'indus_im_y'], axis =1, inplace = True)

In [178]:
df_main.head()

,home_code,host_code,year,industry,distance,scoreproced,minorinvest,enfcontract,paytax,minorinvest_lag,...,border,aff_sales,aff_ex,aff_im,aff_parex,aff_parim,fdi_flow,fdi_position,indus_ex,indus_im
0,USA,ABW,2000,1,3147.434733,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271792324.0,1.535549e+09
1,USA,ABW,2000,2,3147.434733,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71186832.0,1.461237e+09
2,USA,ABW,2000,3,3147.434733,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200605492.0,7.431215e+07
3,USA,ABW,2000,4,3147.434733,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30969497.0,1.302712e+06
4,USA,ABW,2000,5,3147.434733,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23017107.0,3.524300e+04


In [179]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77900 entries, 0 to 77899
Data columns (total 53 columns):
home_code            77900 non-null object
host_code            77900 non-null object
year                 77900 non-null object
industry             77900 non-null object
distance             75620 non-null float64
scoreproced          41980 non-null float64
minorinvest          41980 non-null float64
enfcontract          47720 non-null float64
paytax               41980 non-null float64
minorinvest_lag      38400 non-null float64
uncertainty          32200 non-null float64
home_gdp             73800 non-null float64
host_gdp             67720 non-null float64
couprisk             61200 non-null float64
couprisk_lag         61160 non-null float64
tariff_avg           50900 non-null float64
tariff_w_avg         50900 non-null float64
uncertainty_lag      29900 non-null float64
home_gdp_capita      73800 non-null float64
host_gdp_capita      67720 non-null float64
partner_directe

In [180]:
df_main.to_csv(directory + 'affiliates_dataV2.csv', index = False)